# Car Price Prediction for Kaggle

### **Project Overview**

The goal of this project is to predict the price of used cars based on a variety of their features. This notebook details the process from data loading and cleaning to feature engineering and, finally, model training and evaluation. The project follows an iterative approach, starting with a baseline model and progressively adding features and experimenting with different algorithms to improve prediction accuracy.

### **Table of Contents**
1.  [Setup and Imports](#1.-Setup-and-Imports)
2.  [Data Loading and Initial Exploration](#2.-Data-Loading-and-Initial-Exploration)
3.  [Feature Engineering](#3.-Feature-Engineering)
4.  [Modeling and Experimentation](#4.-Modeling-and-Experimentation)
5.  [Final Model Training and Submission](#5.-Final-Model-Training-and-Submission)

## 1. Setup and Imports

First, we'll import the necessary libraries for data manipulation, modeling, and visualization. We are also setting some `pandas` options for better display of dataframes.

The `eli5` library is used for inspecting machine learning models. If it's not installed in your environment, the following cell will handle the installation.

In [ ]:
# Standard libraries for data analysis
import pandas as pd
import numpy as np
import re

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning models
import xgboost as xgb
import catboost as cb

# Scikit-learn utilities
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# Library for inspecting ML models
try:
    import eli5
except ImportError:
    !pip install -q eli5
    import eli5
from eli5.sklearn import PermutationImportance


# Display options
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

print("Libraries imported successfully!")

## 2. Data Loading and Initial Exploration

We'll load the training and testing datasets. These are provided in HDF5 format. After loading, we will combine them into a single DataFrame to ensure that feature engineering is applied consistently across both datasets.

In [ ]:
# Load the training and test data
# !!! IMPORTANT !!!
# Replace 'path/to/your/df.train.h5' and 'path/to/your/df.test.h5' with the actual file paths.
df_train = pd.read_hdf('df.train.h5')
df_test = pd.read_hdf('df.test.h5')


# Combine the dataframes for consistent processing
df = pd.concat([df_train, df_test], ignore_index=True)

print("Training data shape:", df_train.shape)
print("Test data shape:", df_test.shape)
print("Combined data shape:", df.shape)

df.head()

The target variable, `price_value`, is currently stored as a string with currency information. We need to clean this to have a numerical format for our regression model. We will also take the logarithm of the price, which is a common practice for right-skewed target variables like price, helping the model to perform better.

In [ ]:
# Function to parse the price string into a numeric format
def parse_price(price_str):
    if isinstance(price_str, str):
        return float(price_str.replace('PLN', '').replace(' ', '').replace(',', '.'))
    return float(price_str)

# Apply the parsing function
df['price_value'] = df['price_value'].apply(parse_price)

# Log-transform the target variable to handle skewness
df['price_value_log'] = np.log1p(df['price_value'])

# Visualize the distribution of the original and log-transformed price
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.histplot(df['price_value'], kde=True, bins=50)
plt.title('Distribution of Car Price')

plt.subplot(1, 2, 2)
sns.histplot(df['price_value_log'], kde=True, bins=50)
plt.title('Distribution of Log-Transformed Car Price')

plt.tight_layout()
plt.show()

## 3. Feature Engineering

This is a crucial step where we create new features from the existing data to help our model learn better.

In [ ]:
# Identify all categorical features
categorical_feats = [col for col in df.columns if '_cat' in col]
print(f"Identified {len(categorical_feats)} categorical features: {categorical_feats}")

In [ ]:
# Feature Engineering for Year of Production
df["year_production"] = df["Rok produkcji"].astype(int)
df["year_production_ext"] = df.apply(lambda x: x["year_production"] if x["year_production"] != -1 else int(x["Year"]), axis=1)

In [ ]:
# Feature Engineering for Version
df["version_years"] = df["Wersja"].map(lambda x: re.findall(r'(\d{4})-(\d{4})?', str(x)))
df["version_year_from"] = df["version_years"].map(lambda x: int(x[0][0]) if x else -1)
df["version_year_to"] = df["version_years"].map(lambda x: int(x[0][1]) if x and x[0][1] != "" else -1)

In [ ]:
# Function to remove outliers by capping at a given percentile
def cap_outliers(series, percentile=99):
    cut_value = np.percentile(series[series.notna()], percentile)
    return series.map(lambda x: x if x < cut_value else cut_value)

# Clean and process Engine Capacity
df['engine_capacity'] = df['Pojemność skokowa'].map(lambda x: int(str(x).split('cm3')[0].replace(' ', '')) if pd.notna(x) else -1)
df['engine_capacity_capped'] = cap_outliers(df['engine_capacity'])

# Clean and process Horsepower
df['horse_power'] = df['Moc'].map(lambda x: int(str(x).split('KM')[0].replace(' ', '')) if pd.notna(x) else -1)
df['horse_power_capped'] = cap_outliers(df['horse_power'])

# Clean and process Mileage
df["mileage"] = df["Przebieg"].map(lambda x: int(str(x).split("km")[0].replace(" ", "")) if pd.notna(x) else -1)
df["mileage_capped"] = cap_outliers(df["mileage"])

# Visualize the distributions before and after capping
plt.figure(figsize=(18, 10))

plt.subplot(2, 3, 1)
sns.histplot(df['engine_capacity'], bins=50, kde=True).set_title('Engine Capacity (Original)')
plt.subplot(2, 3, 2)
sns.histplot(df['horse_power'], bins=50, kde=True).set_title('Horse Power (Original)')
plt.subplot(2, 3, 3)
sns.histplot(df['mileage'], bins=50, kde=True).set_title('Mileage (Original)')

plt.subplot(2, 3, 4)
sns.histplot(df['engine_capacity_capped'], bins=50, kde=True).set_title('Engine Capacity (Capped)')
plt.subplot(2, 3, 5)
sns.histplot(df['horse_power_capped'], bins=50, kde=True).set_title('Horse Power (Capped)')
plt.subplot(2, 3, 6)
sns.histplot(df['mileage_capped'], bins=50, kde=True).set_title('Mileage (Capped)')

plt.tight_layout()
plt.show()

In [ ]:
# Create binary feature for 'Serviced at ASO'
df['serviced_at_aso'] = df['Serwisowany w ASO'].apply(lambda x: 1 if x == 'Tak' else 0)

## 4. Modeling and Experimentation

With our features ready, we can now train and evaluate our models. We will use **CatBoost**, a powerful gradient boosting library that works well with categorical features.

In [ ]:
# Model training and evaluation function
def run_model(df, features, model):
    """
    Trains and evaluates a given model using cross-validation.
    
    Args:
        df (pd.DataFrame): The input dataframe containing both train and test sets.
        features (list): A list of feature names to use for training.
        model: The machine learning model to train.
        
    Returns:
        tuple: A tuple containing the mean absolute error and its standard deviation.
    """
    # Filter for training data only (where price is not null)
    train_df = df[df['price_value'].notna()].copy()
    
    X = train_df[features].values
    y = train_df['price_value_log'].values
    
    # Identify categorical feature indices for CatBoost
    cat_feature_indices = [i for i, feat in enumerate(features) if '_cat' in feat]
    
    scores = []
    
    # Cross-validation
    cv = KFold(n_splits=3, shuffle=True, random_state=42)
    for train_idx, test_idx in cv.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        if isinstance(model, cb.CatBoostRegressor):
            model.fit(X_train, y_train, cat_features=cat_feature_indices, eval_set=(X_test, y_test), verbose=0, early_stopping_rounds=50)
        else:
            model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        y_pred = np.expm1(y_pred) # Inverse log-transform
        y_test_exp = np.expm1(y_test)
        
        score = mean_absolute_error(y_test_exp, y_pred)
        scores.append(score)
        
    return np.mean(scores), np.std(scores)

In [ ]:
# Experiment 1: Baseline with basic features
print("Running Experiment 1...")
feats_v1 = ["year_production_ext", "version_year_from", "version_year_to"] + categorical_feats
model = cb.CatBoostRegressor(loss_function="RMSE", random_state=42, verbose=0)
mae, std = run_model(df, feats_v1, model)
print(f"Baseline MAE: {mae:.2f} (+/- {std:.2f})\n")

# Experiment 2: Adding cleaned numerical features
print("Running Experiment 2...")
feats_v2 = feats_v1 + ['engine_capacity_capped', 'horse_power_capped', 'mileage_capped']
model = cb.CatBoostRegressor(loss_function="RMSE", random_state=42, verbose=0)
mae, std = run_model(df, feats_v2, model)
print(f"MAE with numerical features: {mae:.2f} (+/- {std:.2f})\n")

# Experiment 3: Adding the binary ASO feature
print("Running Experiment 3...")
feats_v3 = feats_v2 + ['serviced_at_aso']
model = cb.CatBoostRegressor(loss_function="RMSE", random_state=42, verbose=0)
mae, std = run_model(df, feats_v3, model)
print(f"MAE with ASO feature: {mae:.2f} (+/- {std:.2f})\n")

# Experiment 4: Hyperparameter Tuning
print("Running Experiment 4...")
feats_final = feats_v3 # Use the best feature set so far
tuned_model = cb.CatBoostRegressor(
    loss_function="RMSE", 
    random_state=42, 
    verbose=0,
    iterations=5000,
    learning_rate=0.03
)
mae, std = run_model(df, feats_final, tuned_model)
print(f"MAE with tuned CatBoost: {mae:.2f} (+/- {std:.2f})\n")

## 5. Final Model Training and Submission

Now, we train our final, tuned model on the entire training dataset and make predictions on the test set for submission to Kaggle.

In [ ]:
# Prepare the final feature set
final_features = feats_v3
cat_feature_indices = [i for i, feat in enumerate(final_features) if '_cat' in feat]

# Separate the training and test data
df_train_final = df[df['price_value'].notna()]
df_test_final = df[df['price_value'].isna()]

X_train = df_train_final[final_features].values
y_train = df_train_final['price_value_log'].values
X_test = df_test_final[final_features].values

# Initialize and train the final model
final_model = cb.CatBoostRegressor(
    loss_function="RMSE",
    random_state=42,
    iterations=15000,
    learning_rate=0.03,
    verbose=1000
)

print("Training final model...")
final_model.fit(X_train, y_train, cat_features=cat_feature_indices)

# Make predictions on the test data
print("\nMaking predictions...")
predictions_log = final_model.predict(X_test)
predictions = np.expm1(predictions_log)

# Create the submission file
print("Creating submission file...")
submission = df_test_final[['id']].copy()
submission['price'] = predictions
submission.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
submission.head()

In [ ]:
# Display feature importances
feature_importances = pd.DataFrame(
    {'feature': final_features, 'importance': final_model.get_feature_importance()}
).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 15))
sns.barplot(x='importance', y='feature', data=feature_importances)
plt.title('Feature Importances from Final CatBoost Model')
plt.tight_layout()
plt.show()